In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/Github/SSM/")
# Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia/")

  Activating project at `\Users\ryansenne\Documents\Github\SSM`


In [2]:
include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
# include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using Random
using .SSM
using Distributions
using Plots
using LinearAlgebra
using ForwardDiff
using StatsFuns
using Optim

In [6]:
# generate synthetic data
Random.seed!(123)

# set parameters
n = 100
β = [0.1, 0.9]

# generate data
X = hcat(ones(n), randn(n, 1))

p = logistic.(X * β)
y = rand.(Bernoulli.(p));

logreg = SSM.BernoulliRegression()
SSM.fit!(logreg, reshape(X[:, 2], length(y), 1), y)

2-element Vector{Float64}:
 -0.2254724389883398
  1.1635789534220071

In [63]:
objective_gradient!(zeros(2), [0., 0.])

2-element Vector{Float64}:
 -5.0
 19.769322685726184

In [60]:
res = optimize(objective, objective_gradient!, [0., 0.], LBFGS())
res.minimizer

2-element Vector{Float64}:
 -0.22547243895932628
  1.1635789534102206

In [ ]:
logreg = SSM.BernoulliRegression()
SSM.fit!(logreg, reshape(X[:, 2], length(y), 1), y)

In [7]:
using LinearAlgebra
using StatsFuns: logistic

function loglikelihood(β::AbstractVector, X::Matrix{Float64}, y::Union{Vector{Float64}, BitVector}, w::Float64=1.0)
    # Calculate log likelihood
    p = logistic.(X * β)
    # Convert y if necessary
    y = convert(Vector{Float64}, y)
    return sum(w .* (y .* log.(p) .+ (1 .- y) .* log.(1 .- p)))
end

function gradient(β::AbstractVector, X::Matrix{Float64}, y::Union{Vector{Float64}, BitVector}, w::Float64=1.0)
    # Calculate probabilities
    p = logistic.(X * β)
    # Convert y if necessary
    y = convert(Vector{Float64}, y)
    # Compute the gradient
    return X' * (y .- p) .* w
end

# Example usage with Optim.jl
using Optim

function optimize_loglikelihood(X::Matrix{Float64}, y::Union{Vector{Float64}, BitVector}, w::Float64=1.0)
    # Objective function for Optim
    function obj(β)
        -loglikelihood(β, X, y, w) # we minimize, so take the negative log likelihood
    end

    # Gradient function for Optim
    function grad!(g, β)
        g .= -gradient(β, X, y, w) # we minimize, so take the negative gradient
    end

    # Initial guess for β
    β0 = zeros(size(X, 2))

    # Perform optimization using L-BFGS
    result = optimize(obj, grad!, β0, LBFGS())
    return result
end

# Example data
X = [1.0 1.0; 1.0 2.0; 1.0 3.0]
y = [0.0, 1.0, 1.0]

# Perform optimization
result = optimize_loglikelihood(X, y)
println("Optimal β: ", result.minimizer)

Optimal β: [-57.356469034600124, 31.364423201425744]
